In [6]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Desktop/code/GPEC_hackaton/JPEC`


### Vacuum_math

In [1]:
using SpecialFunctions

#############################################################
# Legendre function of the first kind eq.(47)~(50) , replacing aleg
#############################################################

# Chance eq.(49)
function P0_minus_half(s)
    m1 = 2 / (s + 1)
    return 2 / π * sqrt(m1) * ellipk(m1)
end


# Chance eq.(50)
function P0_plus_half(s)
    m1 = 1 / (s + sqrt(s^2 - 1))^2
    return 2 / π * sqrt(m1) * ellipe(m1)
end


# Chance eq.(48)
function P1_minus_half(s)
    return 1 / ((s^2 - 1)^0.5) * (P0_plus_half(s) - s * P0_minus_half(s))
end


"""
    Pn_minus_half(s, n)

Chance 논문 식 (47)~(50)
calculate Pⁿ_{-1/2}(s) with recursive

# Arguments
- `s::Real` : Legendre function factor (s > 1)
- `n::Int` : maxinum order of n (0 이상)

# Returns
- `P[end]` :  P_{-1/2}^{n}(s) value in nmax 

"""
function Pn_minus_half(s::Real, n::Int)

    #initialize
    P = zeros(n + 1)

    # n = 0
    P[1] = P0_minus_half(s)
    if n == 0
        return P
    end

    # n = 1
    P[2] = P1_minus_half(s)
    if n == 1
        return P
    end

    # n > 1
    for i in 1:n-1
        # Chance eq.(47)
        P[i+2] = -2 * i * s / sqrt(s^2 - 1) * P[i+1] - (i - 0.5)^2 * P[i]
    end

    return P
end


#############################################################
# Green function eq.(36)~(42). replacing green
#############################################################


"""
    green(xs, zs, xt, zt, n)

입력:
- xs, zs: observation points
- xt, zt: source points
- xtp, ztp : ∂X'/∂θ, ∂Z'/∂θ
- n: mode number

반환:
- G :   2π𝒢ⁿ(θ,θ')
- aval :    𝒥 ∇'𝒢ⁿ∇'ℒ
- aval0:    𝒥 ∇'𝒢⁰∇'ℒ
"""
function green(xs, zs, xt, zt, xtp, ztp, n)

    xs2 = xs^2
    xt2 = xt^2
    x_minus2 = (xs - xt)^2
    x_multiple = xs * xt
    ζ = (zs - zt)
    ζ2 = ζ^2

    ρ2 = x_minus2 + ζ2

    # Chance eq.(41) ℛ = R
    R4 = ρ2 * (ρ2 + 4 * x_multiple)
    R2 = sqrt(R4) 
    R = sqrt(R2)

    # Chance eq.(42) 𝘴 = s
    s = (xs2 + xt2 + ζ2) / R2
    
    # Legendre functions for 
    # P⁰ = p0, P¹ = p1, Pⁿ = pn, Pⁿ⁺¹ = pp 
    legendre = Pn_minus_half(s, n+1)
    p0 = legendre[1]
    p1 = legendre[2]
    pp = legendre[end]
    pn = legendre[end-1]

    # Chance eq.(40) 2π⅁ⁿ = G
    gg = 2 * sqrt(π) * gamma(0.5 - n) / R
    G = gg * pn

    # Chance eq.(44)
    # coefficient
    grad_gg = gg / R4
    begin
        # ∂Gⁿ/∂X' = dG_dX
        aval1 = (n * (xs2 + xt2 + ζ2)*(xs2 - xt2 + ζ2) - xt2*(xt2-xs2+ζ2)) * pn
        aval2 = (2.0 * xt * xs * (xs2-xt2+ζ2)) * pp 
        dG_dX = grad_gg * (aval1 + aval2) / xt
        
        # ∂Gⁿ/∂Z' = dG_dZ
        aval3 = (2.0 * n + 1.0) * (xs2 + xt2 + ζ2) * pn
        aval4 = 4.0 * x_multiple * pp
        dG_dZ = grad_gg * (aval3 + aval4) * ζ
        
        # Chance eq.(51) 
        # 𝒥 ∇'𝒢ⁿ∇'ℒ = aval
        # ∂X'/∂θ = xtp, ∂Z'/∂θ = ztp
        aval = -xt * (ztp * dG_dX - xtp * dG_dZ)

        # bval
        bval = G
    end

    # for 𝓃⩵0,  aval0 = 𝒥 ∇'𝒢⁰∇'ℒ 
    dG_dX0 = grad_gg * (2.0 * xt * xs * (xs2-xt2+ζ2)) * p1 - xt2*(xt2-xs2+ζ2) * p0 / xt
    dG_dZ0 = grad_gg * ((xs2 + xt2 + ζ2) * p0 + 4.0 * x_multiple * p1) * ζ
    aval0 = -xt * (ztp * dG_dX0 - xtp * dG_dZ0)

    return aval, aval0, bval
end


#############################################################
# Inverse Fourier transform
#############################################################
"""
    gll = foranv(gil, cs, dth, twopi; m00=0, l00=0, jmax1=nothing, mth=nothing)

Inverse Fourier transform from theta grid to Fourier (l) space.
- gil: θ-grid matrix (size ≥ m00+mth, l00+jmax1)
- cs:  transformation matrix (nths, jmax1)
- dth: grid spacing (Float64)
- m00, l00: starting indices (Fortran offset, usually 0)
- jmax1: number of Fourier modes
- mth: number of theta points

Returns gll: (jmax1, jmax1) matrix.
"""
function foranv(gil, cs, dth ; jmax1=size(cs,2)) 
    gll = zeros(eltype(gil), jmax1, jmax1)
    mth=size(cs,1)
    for l1 in 1:jmax1
        for l2 in 1:jmax1
            acc = zero(eltype(gil))
            for i in 1:mth
                acc += dth * cs[i, l2] * gil[m00 + i, l00 + l1] * 2π
            end
            gll[l2, l1] = acc
        end
    end
    return gll
end



foranv

In [ ]:
green(2.1629189218489446,
      0.49099087247571593,
      1.4309251319336729,
      -1.0909471147721690,
       -0.31581804571944494,
       -0.13298631469337879,
       1)